# Lab Session: การประยุกต์ใช้ deep learning กับภาพถ่ายดาวเทียมเพื่อตรวจจับการใช้ที่ดินโดยอัตโนมัติ

<!-- Semantic Segmentation ของภาพถ่ายดาวเทียมสำหรับการทำแผนที่การใช้ที่ดิน-->

ในส่วนของ Lab นี้ เราจะสำรวจว่า semantic segmentation สามารถใช้เพื่อตรวจจับการเปลี่ยนแปลงการใช้ที่ดินได้อย่างไร โดยเฉพาะอย่างยิ่ง *สวนยาง (ในการระบุ)* ส่วนนี้จะมอบการฝึกจริงในการตรวจจับการเปลี่ยนแปลงอัตโนมัติในการใช้ประโยชน์ที่ดินโดยใช้เครื่องมือฟรีซอฟต์แวร์ เช่น Colab และ QGIS และท่านจะได้เรียนรู้วิธีปรับแต่งโมเดลเพื่อขยายความครอบคลุมไปยังภูมิภาคอื่นๆ

เนื้อหาของส่วนนี้รวมถึง:
1. บทนำสู่ lab session
2. การตรวจจับการเปลี่ยนแปลงโดยใช้ semantic segmentation
   * 2-1 การเตรียมข้อมูล (Data Preprocessing)
   * 2-2 Model training
   * 2-3 การประเมินโมเดลและการประเมินประสิทธิภาพ (Model evaluation and performance assessment)
   * 2-4 การนำโมเดลที่ผ่านการเทรนไปใช้ในพื้นที่เดิมในปีอื่น
   * 2-5 การตรวจจับตำแหน่งที่มีการเปลี่ยนแปลงครั้งใหญ่ที่สุด
3. การขยายความครอบคลุมของโมเดล
   * 3-1 ทดสอบโมเดลในภูมิภาคอื่น
   * 3-2 การเทรนโมเดลสำหรับภูมิภาคที่ใหญ่ขึ้นด้วยชุดข้อมูลใหม่
   * 3-3 การทดสอบโมเดลด้วยชุดข้อมูลใหม่


# ส่วนที่ 0. การติดตั้งเบื้องต้น

นี่คือโค้ดสำหรับการติดตั้งแพ็คเกจที่จำเป็นและชุดข้อมูลสาธิต

In [ ]:
# update apt repositories and install necessary packages
!apt update
!apt install gdal-bin imagemagick parallel bc jq

# install required Python packages
!pip install segmentation-models tensorflow_addons geopandas rasterio scikit-learn matplotlib

# Getting the custom codes for satellite data processing.
!rm -rf semantic_segmentation_training
!git clone https://github.com/GLODAL/semantic_segmentation_training.git

# Getting the demo dataset
!rm -rf *.zip 2-1_img 2-1_patch 2-4_img 3-3_train_img 3_img ann vec
!wget https://glodal-inc.com/downloads/demo_rubber.zip
#!wget https://glodal-inc.com/downloads/demo_paddy.zip
#!wget https://glodal-inc.com/downloads/demo_pineapple.zip
!unzip -o *.zip


# ส่วนที่ 1. บทนำเกี่ยวกับ Semantic Segmentation ของภาพถ่ายดาวเทียมสำหรับการทำแผนที่สวนยาง

[Semantic Segmentation](https://paperswithcode.com/task/semantic-segmentation) คือเทคนิคหนึ่งของ Computer Vision ที่มีหน้าที่ในการแยกส่วนและจำแนกวัตถุแต่ละวัตถุในภาพแยกจากกันโดยกำหนดคลาสในแต่ละจุดพิคเซลว่าเป็นคลาสอะไร เป็นเครื่องมืออันทรงพลังสำหรับใช้วิเคราะห์ภาพถ่ายดาวเทียม เนื่องจากช่วยให้เราสามารถจัดทำแผนที่และติดตามการเปลี่ยนแปลงการใช้ที่ดินได้อย่างแม่นยำเมื่อเวลาผ่านไป ด้วยการใช้ Semantic Segmentation เพื่อตรวจหาการเปลี่ยนแปลงการใช้ที่ดิน ทำให้เราเข้าใจไดนามิกของการใช้ที่ดินได้ดีขึ้น

ใน lab session นี้ เราจะใช้ภาพจาก Planet และ/หรือ Landsat, Google Colab และ QGIS เพื่อทำการแบ่งส่วน(segmentation)และการแสดงภาพ(visualization)

<dl>Planet</dl>
<dd>ชุดข้อมูลภาพถ่ายจาก Planet ที่ใช้ใน lab session นี้ประกอบด้วยภาพสีจริง(true color composite)ที่มีความละเอียดเชิงพื้นที่ 5 เมตร ด้านล่างเป็นตัวอย่างสำหรับนาข้าว ชุดข้อมูลนี้จัดทำขึ้นภายใต้โครงการ NICFI ซึ่งได้รับการสนับสนุนจากรัฐบาลนอร์เวย์ สำหรับข้อมูลเพิ่มเติมเกี่ยวกับชุดข้อมูล โปรดไปที่ <a href="https://www.planet.com/nicfi/">here</a>.<br><img src="https://drive.google.com/uc?export=download&id=1mfWvzqxRQKo12jBHSkyRrwBJ_-OYl2V3" width=320>
</dd>

<dl>Landsat</dl>
<dd>ชุดข้อมูลภาพถ่ายจาก Landsat นิยมใช้ทำแผนที่การใช้ที่ดิน เทคนิคหนึ่งสำหรับสิ่งนี้คือการใช้การผสมสี false color composite เช่น band 564 (รูปภาพด้านล่าง) ซึ่งสามารถช่วยแยกความแตกต่างระหว่างพืชประเภทต่างๆ ด้วยความละเอียดเชิงพื้นที่ 30 เมตร ภาพถ่าย Landsat สามารถเก็บรายละเอียดของสวนยางได้ สำหรับข้อมูลเพิ่มเติมเกี่ยวกับภาพถ่าย Landsat โปรดไปที่ <a href="https://www.usgs.gov/landsat-missions">this website.</a><br><img src="https://drive.google.com/uc?export=download&id=1nMq55ahWwTD7Cd-DE0-hgqiKLSjaT6jh" width=320>
</dd>

<dl>ข้อมูลภาคสนาม Ground truth (GT) dataset</dl>
<dd>ข้อมูลชั้นเวกเตอร์สำหรับสวนยางพาราในรูปแบบโพลีกอนได้รับจากกรมพัฒนาที่ดิน (LDD) สำหรับปีที่เกี่ยวข้อง ชั้นข้อมูลนี้ใช้สำหรับสร้างป้ายกำกับภาพ(labeled images)ของการ training และตรวจสอบความถูกต้องของโมเดล ตลอดจนการประเมินความแม่นยำของผลลัพธ์ที่คาดการณ์ไว้(predicted results) นี่คือผลลัพธ์ที่คาดการณ์ไว้ </dd>

<dl>Google Colab</dl>
<dd>Google Colab เป็นแพลตฟอร์มคลาวด์ที่ไม่มีค่าใช้จ่ายซึ่งเปิดโอกาสให้ผู้ใช้เขียนและรันโค้ด Python ใน Jupyter Notebook environment พร้อมคุณสมบัติดังต่อไปนี้: <br>1) พร้อมสำหรับการเรียนรู้และฝึกฝน machine learning, <br>2) เข้าถึง GPUs ฟรี, <br>3) การผสานรวมกับ Google Drive และการแชร์ที่ง่ายดาย <br>
หากต้องการค้นหาข้อมูลเพิ่มเติมเกี่ยวกับ Google Colab โปรดดู <a href="https://www.youtube.com/watch?v=inN8seMm7UI">this video</a></dd>



<dl>QGIS</dl>
<dd>QGIS เป็นซอฟต์แวร์ GIS ที่ฟรีและเป็น open-source ช่วยให้ผู้ใช้สามารถวิเคราะห์ ดู และแก้ไขข้อมูลเชิงพื้นที่ได้ สำหรับข้อมูลเพิ่มเติมเกี่ยวกับ QGIS หากต้องการดาวน์โหลดและติดตั้ง โปรดไปที่เว็บไซต์ <a href="https://www.qgis.org/en/site/">here.</a></dd>

<dl><a href="https://github.com/qubvel/segmentation_models">Segmentation Models</a></dl>
<dd>ในงานด้านคอมพิวเตอร์ semantic segmentation models ใช้ deep learning algorithms เพื่อระบุและแบ่งส่วนวัตถุภายในภาพในระดับพิกเซล เช่น สวนยางพารา Semantic segmentation models ประเภทต่างๆ เช่น fully convolutional networks (FCN), U-Net, และ Mask R-CNN ต่างมีจุดแข็งและจุดอ่อนของตัวเอง ใน lab session นี้ semantic segmentation model <a href="https://paperswithcode.com/method/fpn">Feature Pynamic Network (FPN)</a> สามารถใช้เพื่อระบุและจัดทำแผนที่พื้นที่สวนยางภายในภาพ Landsat ด้วยการ training model บน labeled data ซึ่งสามารถสร้างขึ้นโดยใช้ข้อมูลภาคสนาม ทำให้สามารถเรียนรู้ที่จะรับรู้ลักษณะเฉพาะของสวนยางและแยกความแตกต่างจากพืชพันธุ์หรือสิ่งปกคลุมดินชนิดอื่นๆได้</dd>

# ส่วนที่ 2. การตรวจจับการเปลี่ยนแปลงโดยใช้ semantic segmentation
ในส่วนนี้ เราจะครอบคลุมขั้นตอนที่เกี่ยวข้องในการเตรียมข้อมูลสำหรับการตรวจจับการเปลี่ยนแปลง, การเทรนโมเดล(training the model) และการประเมินประสิทธิภาพของโมเดล(model's performance)

## 2-1 การเตรียมข้อมูล (Data Pre-processing)
ในส่วนย่อยนี้ เราจะเตรียมภาพถ่ายดาวเทียมและข้อมูลภาคสนาม(GT) สำหรับ semantic segmentation ต่อไปคือวัตถุประสงค์จากส่วนนี้:
1.  ทำความเข้าใจกับรูปแบบ(formats)ของชุดข้อมูล training ซึ่งรวมถึง
  *   Patch images
  *   ช่อง RGB และป้ายกำกับ (RGB chanels and labels)
  *   ตัวอย่างโครงสร้างไฟล์ที่ใช้สำหรับ lab session นี้

2.   ทำความเข้าใจขั้นตอนก่อนการประมวลผลที่จำเป็นในการเตรียมภาพถ่ายดาวเทียมและข้อมูล GT สำหรับ semantic segmentation 


### 2.1.1 การสร้าง patch images สำหรับ model training

ข้อมูลรูปภาพต้องจัดรูปแบบเป็น patche images มีขนาด 64x64, 128x128, หรือ 224x224, ... ตามสมมติฐานอัลกอริธึมของ CNN นอกจากนี้ ข้อมูล GT ของการใช้ที่ดินจัดทำเป็นเลเยอร์เวกเตอร์โพลีกอน ซึ่งไม่เหมาะสำหรับ CNN ดังนั้นจึงต้องใช้กระบวนการที่ยุ่งยากเล็กน้อยเพื่อใช้ใน CNN

เนื่องจากโมเดล semantic segmentation ต้องการข้อมูลอินพุตในรูปแบบของภาพราสเตอร์, เราจำเป็นต้องแปลงข้อมูลเวกเตอร์เป็นรูปแบบภาพราสเตอร์ ภาพราสเตอร์จะมีขนาดและความละเอียดเดียวกันกับภาพถ่ายดาวเทียมอินพุต โดยแต่ละพิกเซลจะกำหนดค่าเป็น 1 หรือ 0 ขึ้นอยู่กับว่าภาพนั้นอยู่ในบริเวณที่สนใจหรือไม่ โค้ดต่อไปนี้สาธิตวิธีการแปลงเป็นไฟล์ราสเตอร์

ในภาคปฏิบัตินี้ เราได้กำหนด patch size เป็น 128x128 ซึ่งเป็นขนาดที่เพียงพอในการจับภาพลักษณะการใช้ที่ดิน โดยทั่วไปแล้ว ขนาดใหญ่กว่าย่อมดีกว่า แต่ patch size จะถูกจำกัดโดยทรัพยากรในการคำนวณ เช่น RAM หรือหน่วยความจำ

In [ ]:
# Configure parameters for patch images.
n_patch = 50     # number of patches per image file. For this practice n_patch ~ 200 will be preferred.
                 # Since the patch generation processes five types of data augumentation (vertical flip, horizental flip, and rotation 90, 180, and 270 degrees) for every patches, actual number of patches are six times of this paramter.
patch_size = 128 # size of patch images.

in_vec_dir = 'vec/'
in_img_dir = '2-1_img/'
out_ann_dir = 'ann/'
out_patch_dir = '2-1_patch'

import os
# The module is prepared in shell script. You can see detail by browsing the file inside or https://github.com/GLODAL/semantic_segmentation_training/blob/main/rasterize_polygon_layers.sh
os.system('bash semantic_segmentation_training/rasterize_polygon_layers.sh "'
          + in_vec_dir + '" "'
          + in_img_dir + '" "'
          + out_ann_dir +'"'
          )


ต่อไป เราจะสร้างชุดของ patches จากภาพราสเตอร์และคำอธิบายประกอบที่เกี่ยวข้อง

In [ ]:
import os
in_ann_dir = out_ann_dir
# The module is prepared in shell script. You can see detail by browsing the file inside or https://github.com/GLODAL/semantic_segmentation_training/blob/main/gen_training_patches.sh
os.system('bash semantic_segmentation_training/gen_training_patches.sh "' 
         + in_img_dir + '" "' 
          + in_ann_dir + '" ' 
          + str(patch_size) + ' '
          + str(n_patch) + ' '
          + out_patch_dir
          )


หลังจากสร้างข้อมูล patch แล้ว ท่านจะพบไฟล์ patch image ในโฟลเดอร์ ใต้ '2-1_patch' พร้อมด้วยโฟลเดอร์ย่อย 'patch_img' และ 'patch_ann' สำหรับภาพดาวเทียมและข้อมูล label จากชั้นการใช้ประโยชน์ที่ดิน ตามลำดับ

นี่คือตัวอย่างของ patch ของรูปภาพและคำอธิบายประกอบที่เกี่ยวข้องซึ่งสร้างขึ้นโดยใช้โค้ดด้านบน
<br><img src="https://drive.google.com/uc?export=download&id=1nUWSRHlnTqbtXllkNuawClpSQLSrbqJM" width=640>

หากต้องการเรียกดูผลลัพธ์ ให้ทำตามขั้นตอนด้านล่าง:
1. คลิกไอคอน "Files" ที่แผงด้านขวา 
2. หลายโฟลเดอร์จะปรากฏ:
  - 2-1_image: ภาพ Landsat สำหรับชุดข้อมูล training ของปีฐาน
  - 2-1_patch: Patch image และคำอธิบายประกอบที่สร้างใน 2_1
  - 2-4_image: ภาพ Landsat สำหรับชุดข้อมูล testing ของปีที่ต้องการทำนาย
  - 3-3_train_img: ภาพ Landsat สำหรับชุดข้อมูล training ของสองภูมิภาคสำหรับปีฐาน
  - 3_img: ภาพ Landsat สำหรับชุดข้อมูล testing ของปีที่ต้องการทำนาย
  - ann: ชั้นการใช้ประโยชน์ที่ดินราสเตอร์ใน 2-1
  - semantic_segmentation_training: รหัสแบบกำหนดเองสำหรับการประมวลผลข้อมูลดาวเทียมที่ดาวน์โหลดในส่วนที่ 0 
  - vec: Polygon layer ของคลาสย่อยจากชั้นการใช้ที่ดินที่จัดทำโดย LDD
3. เลือกโฟลเดอร์ชื่อ **2-1_patch** มีโฟลเดอร์ย่อยสองโฟลเดอร์ภายในโฟลเดอร์นี้ชื่อ **patch_ann** และ **patch_image** สิ่งเหล่านี้บ่งบอกถึง patch ของ label data และภาพถ่ายดาวเทียม
4. ภายในโฟลเดอร์ย่อย **patch_ann** หรือ **patch_image** สำหรับ หนึ่ง patch image จะมีไฟล์ TIFF หกไฟล์ที่แสดงแทนไฟล์ TIFF ดั้งเดิม การหมุน 90, 180 และ 270 องศา พลิก และปัด สิ่งเหล่านี้ถูกสร้างขึ้นโดยใช้เทคนิคการเสริมข้อมูลเพื่อเพิ่มจำนวน training data
5. คลิกขวาที่รูปภาพ TIFF ใดก็ได้และดาวน์โหลด **โปรดแน่ใจว่าคุณดาวน์โหลดภาพ TIFF ด้วยชื่อเดียวกันกับไฟล์คำอธิบายประกอบที่เกี่ยวข้อง**
6. นำเข้ารูปภาพที่ดาวน์โหลดมาใน QGIS เพื่อให้แสดงภาพ

เพื่ออธิบายขั้นตอนข้างต้น โปรดดูรูปด้านล่าง
<br><img src="https://drive.google.com/uc?export=download&id=1nW-p4kP2fGC5Dd4qTDdMnM4VPUbjDdXh" width=640>



## 2.2 Model training
ในส่วนย่อยนี้ เราจะมุ่งเน้นไปที่การเทรนนิ่ง semantic segmentation โดยใช้ข้อมูลที่เตรียมไว้จากหัวข้อ นี่คือประเด็นหลักจากส่วนนี้:
* ทำความเข้าใจส่วนประกอบของ model training สำหรับ semntic segmentation
* ทำความเข้าใจการจัดการ model training เพื่อการใช้งานจริง
* ทำความเข้าใจกับกรณีความล้มเหลวทั่วไประหว่าง model training และตัวเลือกที่มีอยู่สำหรับการปรับปรุงกระบวนการ model training

ในเซสชันแล็บนี้ เราใช้ [FPN](https://paperswithcode.com/method/fpn) model เป็นเครือข่ายประสาทเทียมแบบเต็มรูปแบบ (convolutional neural network) ที่ใช้สำหรับ binary segmentation เช่น การจำแนกพิกเซลในส่วนหน้าและพื้นหลัง ท่านสามารถดูรายละเอียดทางทฤษฎีได้จากลิงค์ด้านบน FPN มีประสิทธิภาพที่ได้เปรียบในแอปพลิเคชันสำหรับการสำรวจระยะไกลผ่านดาวเทียมมากกว่า UNet โมเดลที่นิยมมากที่สุดใน semantic segmentatoin

### 2.2.1 การสร้าง image generator สำหรับกระบวนการ batch 
วัตถุประสงค์ของส่วนย่อยนี้คือการกำหนดฟังก์ชันสำหรับการสร้าง batch ซึ่งป้อน patch images ไปยัง model training ท่านสามารถดูขั้นตอนการ
การสร้าง batched ของภาพถ่ายดาวเทียมและ label image ที่เกี่ยวข้องสำหรับ training และ testing ในฟังก์ชั่น image_generator

FYI: Keras จัดเตรียมฟังก์ชัน [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) เพื่อจุดประสงค์เดียวกัน แต่ฟังก์ชันนี้ไม่เหมาะสำหรับ semantic segmentation โดยใช้ชุดข้อมูลของพวกเรา

In [ ]:
validation_split = 0.2 # Float. Fraction of images reserved for validation (strictly between 0 and 1).
batch_size = 32

import numpy as np, random, glob
from PIL import Image

# generating batches of images and their corresponding masks
def image_generator(files, batch_size = batch_size, sz = (patch_size, patch_size)):
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    
    for f in batch:
        #get the masks. Note that masks are png files 
        try:
          mask = Image.open(f.replace("patch_img","patch_ann"))
        except:
          continue
        mask = np.array(mask.resize(sz))
        batch_y.append(mask)

        #preprocess the raw images 
        raw = Image.open(f)
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
          raw = np.stack((raw,)*3, axis=-1)
        else:
          raw = raw[:,:,0:3]
        
        batch_x.append(raw)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255.
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,axis = 3)

    yield (batch_x, batch_y.astype(np.float32))

# list of all files in the directory specified by patch_img
all_files = glob.glob(out_patch_dir + '/patch_img/*.tif')

# randomly shuffles the order of the files in all_files
random.shuffle(all_files) 

#split into training and testing
split = int((1-validation_split) * len(all_files))
train_files = all_files[0:split]
test_files  = all_files[split:]

train_generator = image_generator(train_files, batch_size = batch_size) # creates a generator for the training set 
test_generator  = image_generator(test_files, batch_size = batch_size) # creates a generator for the testing set 

# retrieves the next batch of images and masks from the training set generator
x, y= next(train_generator)


โค้ดด้านล่างจะสร้างพล็อตรูปภาพที่แสดงรูปภาพอินพุตและคำอธิบายประกอบที่เกี่ยวข้อง เราอาจจะเปลี่ยน array index เพื่อดูตัวอย่างอื่นๆ

In [ ]:
import matplotlib.pyplot as plt

# generates an image plot of an input image and its corresponding mask
plt.axis('off')
i = 1
img = x[i]
msk = y[i].squeeze()
msk = np.stack((msk,)*3, axis=-1)
print(img.shape)

plt.imshow(np.concatenate([img, msk, img*msk], axis = 1))

### 2.2.2 กำลังเตรียมโมเดลที่จะเทรน

ส่วนนี้เป็นการเตรียมโมเดลโดยอ้างอิงโมเดลที่เทรนไว้ล่วงหน้าซึ่งจัดทำโดย [Segmentation Models](https://github.com/qubvel/segmentation_models) ส่วนนี้มีพารามิเตอร์หลักสำหรับ model training (หรือที่เรียกว่าไฮเปอร์พารามิเตอร์) ดังนั้นท่านอาจกลับมาที่ส่วนนี้เพื่อปรับแต่ง model training หากโมเดลล้มเหลว

In [ ]:
backbone = 'efficientnetb1'

os.environ['SM_FRAMEWORK'] = "tf.keras"
import segmentation_models as sm
import tensorflow_addons as tfa

model = sm.FPN(backbone, classes=1, activation='sigmoid')
model.compile(
    optimizer = tfa.optimizers.RectifiedAdam(),
    loss = sm.losses.bce_jaccard_loss,
    metrics = ['accuracy',sm.metrics.iou_score]
)

### 2.2.3 การสร้างฟังก์ชั่น callback 

เรากำหนดฟังก์ชัน callback ซึ่งเรียกใช้ทุกจุดเริ่มต้นหรือจุดสิ้นสุดของ epoch เพื่อการจัดการกระบวนการ training ที่ดีขึ้น เรากำหนดสองฟังก์ชัน callback

1. การเซฟโมเดลที่ epochs มีประสิทธิภาพสูงสุด สิ่งนี้มีประโยชน์สำหรับการเก็บโมเดลเพื่อใช้กับชุดข้อมูลอื่น ไม่เพียงแต่ใน model training เท่านั้น ดู [คู่มือผู้ใช้ Keras](https://keras.io/api/callbacks/model_checkpoint/) สำหรับรายละเอียดเพิ่มเติม
2. แสดงผล testing ทุก epoch TensorFlow แสดง performance metrics ในทุกช่วงท้ายของ epoch แต่บางครั้งก็ไม่มีประโยชน์สำหรับวิธีที่โมเดลแสดงผลลัพธ์ของการทำนาย สิ่งนี้ช่วยให้เข้าใจโดยสังเขปเกี่ยวกับความคืบหน้าใน model training




In [ ]:
checkpoint_file = '2-2_checkpoint.hdf5'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_file, save_weights_only=True, save_best_only=True)

# inheritance for training process plot 
class PlotLearning(tf.keras.callbacks.Callback):

    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        #self.fig = plt.figure()
        self.logs = []
        
    def on_epoch_end(self, epoch, logs={}):
        
        #choose a random test image and preprocess
        path = np.random.choice(test_files)
        raw = Image.open(path)
        raw = np.array(raw)/255.
        #print(raw.shape)
        raw = raw[:,:,0:3]
        
        #predict the mask 
        pred = model.predict(np.expand_dims(raw, 0))
        
        #mask post-processing 
        msk  = pred.squeeze()
        msk = np.stack((msk,)*3, axis=-1)
        msk[msk >= 0.5] = 1 
        msk[msk < 0.5] = 0 
        
        #show the mask and the segmented image 
        combined = np.concatenate([raw, msk, raw* msk], axis = 1)
        plt.axis('off')
        plt.imshow(combined)
        plt.show()

### 2-2-4 Training
ขั้นตอนสุดท้ายของส่วนนี้คือการปฏิบัติ model training โดยใช้ชุด training และประเมินประสิทธิภาพโดยใช้ชุด testing

In [ ]:
n_epoch = 20

import math, glob
train_step = math.ceil(len(glob.glob(out_patch_dir+'/patch_img/*.tif')) * (1 - validation_split) / batch_size)
test_step = math.ceil(len(glob.glob(out_patch_dir+'/patch_img/*.tif')) * validation_split / batch_size)

model_history = model.fit(train_generator, 
                          epochs = n_epoch, 
                          steps_per_epoch = train_step,
                          validation_data = test_generator, 
                          validation_steps = test_step,
                          callbacks = [checkpointer,PlotLearning()], 
                          verbose = 1)


หลังจาก model training ลองพล็อต learning curves เพื่อประเมินความถูกต้องของ model training

In [ ]:
# creating a plot of the training and validation loss over the number of epochs
# extract the training loss and validation loss values 
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

# creates a range object for the number of epochs
epochs = range(n_epoch)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([np.min(np.hstack([loss, val_loss])), np.max(np.hstack([loss, val_loss]))])
plt.legend()
plt.show()

หาก learning curve ล้มเหลวใน overfitting ซึ่ง validation loss เพิ่มขึ้นตามช่วงเวลาเช่นรูปด้านล่าง model จะทำงานได้ไม่ดีนักเนื่องจาก model ถูก localized มากเกินไปใน model training ในขณะที่ชุดข้อมูลอื่น ๆ เหมาะสมแค่เพียงเล็กน้อย

เราขอแนะนำให้เปลี่ยนพารามิเตอร์ดังต่อไปนี้:

1. เพิ่มจำนวน patches ใน <a href="https://colab.research.google.com/drive/1GdXkRyta2DU0uh5TwsqYw8bUF4gHHoEn?authuser=4#scrollTo=IkPpgKoZZ9mM&line=7&uniqifier=1">Section 2-1</a>. รูปแบบภาพที่หลากหลายมากขึ้นใน training data มักจะช่วยให้ model รับรู้ถึงคุณลักษณะที่ละเอียดอ่อนของการใช้ที่ดินได้ อย่างไรก็ตาม อาจใช้เวลานานขึ้นใน model training ท่านควรพิจารณาการแลกเปลี่ยนระหว่างเวลาและประสิทธิภาพ
2. เปลี่ยน backbone ของ model. The backbone ตั้งอยู่ใน <a href="https://colab.research.google.com/drive/1GdXkRyta2DU0uh5TwsqYw8bUF4gHHoEn?authuser=4#scrollTo=SdRssoe4mVA_&line=3&uniqifier=1">Section 2-2-2</a>, "efficientnetb2" มีชื่อเสียงในด้าน robustness จน overfitting อย่างไรก็ตาม ท่านสามารถแทนที่ด้วย backbone ที่ซับซ้อนน้อยกว่าเช่น "efficiencynetb0" และ "resnet50" ท่านสามารถค้นหาตัวเลือกใน [the website of Segmentation Models](https://github.com/qubvel/segmentation_models#models-and-backbones).
3. ลด batch size. โดยทั่วไป, [batch sizes ขนาดใหญ่อาจเป็นสาเหตุเกิด overfitting.](https://stats.stackexchange.com/questions/266368/deep-learning-why-does-increase-batch-size-cause-overfitting-and-how-does-one-r) คุณสามารถกำหนดค่า batch_size ได้ใน [Section 2-2-1](https://colab.research.google.com/drive/1GdXkRyta2DU0uh5TwsqYw8bUF4gHHoEn?authuser=4#scrollTo=HdCG-d0SkHMW). อย่างไรก็ตาม อาจใช้เวลานานขึ้นใน model training ท่านควรพิจารณาการแลกเปลี่ยนระหว่างเวลาและประสิทธิภาพ

|Good model training|Failure in overfitting|
|-------------------|----------------------|
|<img src="https://machinelearningmastery.com/wp-content/uploads/2018/12/Example-of-Train-and-Validation-Learning-Curves-Showing-A-Good-Fit.png" width="640">|<img src="https://machinelearningmastery.com/wp-content/uploads/2018/12/Example-of-Train-and-Validation-Learning-Curves-Showing-An-Overfit-Model.png" width="640">|

Source: <a href="https://machinelearningmastery.com/learning-curves-for-diagnosing-machine-learning-model-performance/">How to use Learning Curves to Diagnose Machine Learning Model Performance</a>


## 2-3 การประเมินโมเดล
จะได้เรียนรู้เกี่ยวกับ:
*  ทำความเข้าใจการประเมินโมเดลในทางปฏิบัติโดยใช้เมตริก เช่น IoU, Learning Curve และ overall accuracy และการตีความตัวบ่งชี้

model training ใช้เฉพาะข้อมูลที่สุ่มตัวอย่างเท่านั้น เนื่องจากข้อมูลการสำรวจระยะไกลโดยทั่วไปมีขนาดใหญ่เกินกว่าจะโหลดข้อมูลทั้งหมดไปยังกระบวนการ เราต้องใช้โมเดลที่ผ่านการเทรนกับขอบเขตทั้งหมดของภาพถ่ายดาวเทียมเพื่อการประเมินความแม่นยำ

เราฝึก testing โมเดลที่ผ่านการเทรนใน 2.2 โดยคำนวณ confusion matrix ระหว่างผลการทำนายและชั้นการใช้ประโยชน์ที่ดิน

### 2-3-1 การนำโมเดลไปใช้กับชุดข้อมูลรูปภาพ

ภาพถ่ายดาวเทียมจะต้องถูกแบ่งออกเป็น patches มีขนาดพิกเซล, multiples of patch images สำหรับชุดข้อมูล training เราแบ่งภาพถ่ายดาวเทียมออกเป็น patchesมีขนาด 256x256 สำหรับอินพุตไปยัง trained model.  

ขั้นแรก เราตั้งค่า model trained in 2-2 น 2-2 เป็นเฟรมเวิร์ก Segmantation Models

In [ ]:
## Specify the model trained in Section 2-2 ##
model_file = '2-2_checkpoint.hdf5'
##############################################

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

# Set a model architecture and backbone same to the model for loading.
model = sm.FPN('efficientnetb2', classes=1, activation='sigmoid')
# Set a path to the trained model file for testing.
model.load_weights(model_file)

จากนั้น ใช้โมเดลที่โหลดกับชุดข้อมูลรูปภาพจากภูมิภาคอื่น ไฟล์สคริปต์ *split_images_for_testing.sh* มีไว้สำหรับแยกภาพถ่ายดาวเทียม และ *merge_annotation_for_testing.sh* มีไว้สำหรับการรวม prediction outputs. ท่านสามารถดูรายละเอียดได้ที่ที่เก็บ GitHub

In [ ]:
input_geotiff = '2-1_img/Prachuap A, Year 2018.tif'
output_pred_file = "test_results/2-3-1 Prachuap A, Year 2018.tif.pred.tif"
test_patch_size = 256

import os, tempfile, shutil, glob
import numpy as np
from PIL import Image
os.environ['SM_FRAMEWORK'] = "tf.keras"
import segmentation_models as sm

workdir = tempfile.mkdtemp()
os.system('bash semantic_segmentation_training/split_images_for_testing.sh "'  # https://github.com/GLODAL/semantic_segmentation_training/blob/main/split_images_for_testing.sh 
          + input_geotiff + '" ' 
          + workdir + ' ' 
          + str(test_patch_size) )

os.makedirs(workdir + '/pat_prd/', exist_ok=True)
for f in glob.glob(workdir + '/pat_img/*.tif'):
  raw = Image.open(f)
  out_patch_size = raw.size[0]
  raw = np.array(raw)
  raw = np.array(raw)/255.
  raw = raw[:,:,0:3]

  pred = model.predict(np.expand_dims(raw, 0))
  pred[pred >= 0.5] = 1
  pred[pred < 0.5] = 0
  pred = pred.astype(np.uint8).reshape([out_patch_size,out_patch_size])
  im = Image.fromarray(pred)
  im.save(workdir + '/pat_prd/' + os.path.basename(f) + '.pred.tif')

os.system('bash semantic_segmentation_training/merge_annotation_for_testing.sh ' # https://github.com/GLODAL/semantic_segmentation_training/blob/main/merge_annotation_for_testing.sh
          + workdir + ' "' 
          + input_geotiff + '" "' 
          + output_pred_file + '"')


ตอนนี้ ท่านสามารถค้นหาผล testing ภาพถ่ายดาวเทียมได้ที่โฟลเดอร์ *test_results* ท่านสามารถเรียกดูผลลัพธ์ได้โดยดาวน์โหลด GeoTiffs (*.tif) บน QGIS ด้านล่างนี้คือตัวอย่างการแสดงผลการทดสอบใน QGIS

*ใส่ตัวอย่างแสดงผลการทดสอบ*

### 2-3-2 การประเมินความแม่นยำโดย by confusion matrix

เราคำนวณ confusion matrix ระหว่าง prediction result และชั้นการใช้ที่ดิน confusion matrix และชั้นการใช้ที่ดิน cross-tabulate ของทั้งสองเลเยอร์ ที่นี่เราฝึกฝนในเซลล์ด้านล่าง

In [ ]:
ground_truth_file = "vec/Rubber plantation, Year 2018.gpkg"
prediction_result_file = "test_results/2-3-1 Prachuap A, Year 2018.tif.pred.tif"

import geopandas as gpd
import rasterio
from rasterio import features
from sklearn import metrics

ground_truth = gpd.read_file(ground_truth_file)
geom = [shapes for shapes in ground_truth.geometry]
prediction_result = rasterio.open(prediction_result_file)

ground_truth_rasterized = features.rasterize(geom,
                                out_shape = prediction_result.shape,
                                fill = 0,
                                out = None,
                                transform = prediction_result.transform,
                                all_touched = False,
                                default_value = 1,
                                dtype = None)

ground_truth_rasterized_1d_array = ground_truth_rasterized.ravel()
prediction_result_1d_array = prediction_result.read(1).ravel()

confusion_matrix = metrics.confusion_matrix(ground_truth_rasterized_1d_array, prediction_result_1d_array)
iou =  confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[0,1] + confusion_matrix[1,0])
print("IoU:       %.4f" % iou)

print("Accuracy:  %.4f" % metrics.accuracy_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))
print("Precision: %.4f" % metrics.precision_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))
print("Recall:    %.4f" % metrics.recall_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))


## 2.4. ใช้ trained model กับภาพถ่ายดาวเทียมล่าสุด

วัตถุประสงค์ของส่วนนี้คือเพื่ออธิบายว่าสามารถนำ pre-trained model กลับมาใช้ใหม่เพื่อสร้างการคาดการณ์ได้อย่างไร โดยเฉพาะอย่างยิ่งสำหรับการสร้างแผนที่นาข้าวใหม่ ประเด็นการเรียนรู้ที่สำคัญ ได้แก่ การทำความเข้าใจถึงประโยชน์ของการใช้ pre-trained models สำหรับการจัดทำแผนที่การใช้ที่ดิน ตลอดจนข้อดีและข้อจำกัดของการผลิตแผนที่ตามโมเดล

In [ ]:
## Specify the model trained in Section 2-2 ##
model_file = '2-2_checkpoint.hdf5'
##############################################

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

# Set a model architecture and backbone same to the model for loading.
model = sm.FPN('efficientnetb2', classes=1, activation='sigmoid')
# Set a path to the trained model file for testing.
model.load_weights(model_file)

จากนั้นนำภาพถ่ายดาวเทียมมาประมวลผลเพื่อใช้เป็นโมเดลเหมือนข้อ 2-3

In [ ]:
input_geotiff = '2-4_img/Prachuap A, Year 2020.tif'
output_pred_file = "test_results/2-4 Prachuap A, Year 2020.tif.pred.tif"
test_patch_size = 256

import os, tempfile, shutil, glob
import numpy as np
from PIL import Image
os.environ['SM_FRAMEWORK'] = "tf.keras"
import segmentation_models as sm

workdir = tempfile.mkdtemp()
os.system('bash semantic_segmentation_training/split_images_for_testing.sh "'  # https://github.com/GLODAL/semantic_segmentation_training/blob/main/split_images_for_testing.sh 
          + input_geotiff + '" ' 
          + workdir + ' ' 
          + str(test_patch_size) )

os.makedirs(workdir + '/pat_prd/', exist_ok=True)
for f in glob.glob(workdir + '/pat_img/*.tif'):
  raw = Image.open(f)
  out_patch_size = raw.size[0]
  raw = np.array(raw)
  raw = np.array(raw)/255.
  raw = raw[:,:,0:3]

  pred = model.predict(np.expand_dims(raw, 0))
  pred[pred >= 0.5] = 1
  pred[pred < 0.5] = 0
  pred = pred.astype(np.uint8).reshape([out_patch_size,out_patch_size])
  im = Image.fromarray(pred)
  im.save(workdir + '/pat_prd/' + os.path.basename(f) + '.pred.tif')

os.system('bash semantic_segmentation_training/merge_annotation_for_testing.sh ' # https://github.com/GLODAL/semantic_segmentation_training/blob/main/merge_annotation_for_testing.sh
          + workdir + ' "' 
          + input_geotiff + '" "' 
          + output_pred_file + '"')


ตอนนี้ ท่านสามารถค้นหาtesting result ภาพถ่ายดาวเทียมได้ที่โฟลเดอร์ *test_results* ท่านสามารถเรียกดูผลลัพธ์ได้โดยดาวน์โหลด GeoTiffs (*.tif) บน QGIS ด้านล่างนี้คือตัวอย่างการแสดงผลการทดสอบใน QGIS
เนื่องจากเรามีชั้นการใช้ประโยชน์ที่ดินสำหรับปี 2021 ท่านจึงสามารถประเมินความแม่นยำของการคาดการณ์สำหรับปี 2021 โดยเปรียบเทียบกับชั้นการใช้ประโยชน์ที่ดินผ่านกระบวนการเช่นเดียวกับ 2-3-2 เราปล่อยให้เป็นแบบฝึกหัดเสริมในเซสชันนี้

## 2.5. ตรวจหาตำแหน่งที่มีการเปลี่ยนแปลงครั้งใหญ่ที่สุด
ส่วนย่อยนี้มีจุดมุ่งหมายเพื่อให้เข้าใจแนวคิดของการตรวจจับการเปลี่ยนแปลงและวิธีนำเทคนิค deep learning มาใช้เพื่อตรวจจับการเปลี่ยนแปลงการใช้ที่ดิน

ประเด็นการเรียนรู้ที่สำคัญของส่วนย่อยนี้ ได้แก่ :
*   ระบุการเปลี่ยนแปลงที่จะตรวจจับได้
*   ทำความเข้าใจเกี่ยวกับคุณภาพของ outputs และวิธีจัดการเพื่อตรวจจับสถานที่ที่มีการเปลี่ยนแปลงการใช้ที่ดินอย่างมีนัยสำคัญ

ส่วนย่อยนี้จะดำเนินการโดยใช้ QGIS



การวิเคราะห์จะต้องเป็นโพลีกอนเพื่อตรวจจับการใช้ประโยชน์ที่ดินที่มีการเปลี่ยนแปลงมากที่สุด เราแปลงผลการทำนายเป็นโพลีกอนในเซลล์ด้านล่าง

In [ ]:
baseline_land_use_file = "vec/Rubber plantation, Year 2018.gpkg"
predicted_land_use_raster_file = "test_results/2-4 Prachuap A, Year 2020.tif.pred.tif"
predicted_land_use_vector_file = "test_results/2-4 Prachuap A, Year 2020.geojson"
import os
os.system('gdal_polygonize.py "'
        + predicted_land_use_raster_file
        + '" -f GeoJSON "'
        + predicted_land_use_vector_file + '"'
        )






In [ ]:
import geopandas

gpd_predicted_land_use = geopandas.read_file(predicted_land_use_vector_file)
gpd_baseline_land_use  = geopandas.read_file(baseline_land_use_file).clip(gpd_predicted_land_use.total_bounds)
gpd_predicted_land_use = gpd_predicted_land_use[gpd_predicted_land_use['DN']==1]


In [ ]:

gpd_baseline_land_use.overlay(gpd_predicted_land_use, how='union')





In [ ]:
!gdal_polygonize.py "test_results/2-4 Prachuap A, Year 2020.tif.pred.tif" -f GeoJSON test.geojson

# ส่วนที่ 3: การถ่ายทอดการเรียนรู้ของโมเดลที่ผ่านการเทรน 

ส่วนนี้ประกอบด้วยแนวทางปฏิบัติของการถ่ายโอนการเรียนรู้ ซึ่งเป็นเทคนิคสำหรับการใช้โมเดลที่เคยผ่านการเทรนซ้ำกับโดเมนอื่นๆ เพื่อประหยัดเวลาและทรัพยากรแทนการฝึกโมเดลตั้งแต่เริ่มต้น ส่วนนี้มีโครงสร้างโดยมีวิธีปฏิบัติดังต่อไปนี้

- 3-1 ทดสอบโมเดลที่ได้รับการเทรนไปยังภูมิภาคอื่นๆ
- 3-2 ปรับแต่งโมเดลที่ได้รับการเทรนไปยังภูมิภาคอื่นๆ
- 3-3 เทรนโมเดลโดยรวมกับชุดข้อมูลการเทรน

หลังจากจบส่วนนี้ท่านจะมีความเข้าใจในข้อจำกัดของโมเดลที่ผ่านการเทรนและขั้นตอนที่จำเป็นสำหรับการใช้โมเดลที่ผ่านการเทรนในขอบเขตทางภูมิศาสตร์ที่กว้างขึ้น

ในส่วนนี้ เรากำหนดเงื่อนไขของ "ภูมิภาค A" และ "ภูมิภาค B" ดังนี้:

- ภูมิภาค A: ภูมิภาคของชุดข้อมูลที่ใช้ในโมเดลเทรนนิ่ง
- ภูมิภาค B: ภูมิภาคของชุดข้อมูลสำหรับการทดสอบโมเดลที่ได้รับการเทรนสำหรับภูมิภาค A

## 3-1  ทดสอบโมเดลที่ผ่านการเทรนไปยังภูมิภาคอื่นๆ

ในส่วนย่อยนี้ ท่านมีการทดลองเล็กๆ ในการทดสอบโมเดลที่ผ่านการเทรนกับชุดข้อมูลอื่นที่ไม่ใช่ชุดข้อมูลการเทรน ท่านจะเห็นประสิทธิภาพที่ไม่ดีของโมเดล เนื่องจากโมเดลถูกจำกัดไปยังภูมิภาคของชุดข้อมูลการเทรน

ขั้นแรก ให้โหลดโมเดลที่ผ่านการเทรนซึ่งสร้างในส่วนที่ 2-2

In [ ]:
## Specify the model trained in Section 2-2 ##
model_file = 'demo_model.hdf5'
##############################################

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm
import tensorflow_addons as tfa
sm.set_framework('tf.keras')

# Set a model architecture and backbone same to the model for loading.
model = sm.FPN('efficientnetb2', classes=1, activation='sigmoid')
# Set a path to the trained model file for testing.
model.load_weights(model_file)

ต่อไป ใช้โมเดลที่โหลดมากับชุดข้อมูลรูปภาพจากภูมิภาคอื่น

In [ ]:
input_geotiff = '3_img/Prachuap B, Year 2018.tif'
output_pred_file = "test_results/3-1 Prachuap B, Year 2018.tif.pred.tif"
test_patch_size = 256

import os, tempfile, shutil, glob
import numpy as np
from PIL import Image
os.environ['SM_FRAMEWORK'] = "tf.keras"
import segmentation_models as sm

workdir = tempfile.mkdtemp()
os.system('bash semantic_segmentation_training/split_images_for_testing.sh "' 
          + input_geotiff + '" ' 
          + workdir + ' ' 
          + str(test_patch_size) )

os.makedirs(workdir + '/pat_prd/', exist_ok=True)
for f in glob.glob(workdir + '/pat_img/*.tif'):
  raw = Image.open(f)
  out_patch_size = raw.size[0]
  raw = np.array(raw)
  raw = np.array(raw)/255.
  raw = raw[:,:,0:3]

  pred = model.predict(np.expand_dims(raw, 0))
  pred[pred >= 0.5] = 1
  pred[pred < 0.5] = 0
  pred = pred.astype(np.uint8).reshape([out_patch_size,out_patch_size])
  im = Image.fromarray(pred)
  im.save(workdir + '/pat_prd/' + os.path.basename(f) + '.pred.tif')

os.system('bash semantic_segmentation_training/merge_annotation_for_testing.sh ' 
          + workdir + ' "' 
          + input_geotiff + '" "' 
          + output_pred_file + '"')


จากนั้นคำนวณ perfomance index เช่น ความแม่นยำ(accuracy) และ IoU เพื่อเปรียบเทียบกับผลการทดสอบในส่วนที่ 2-3

In [ ]:
ground_truth_file = "vec/Rubber plantation, Year 2018.gpkg"
prediction_result_file = "test_results/3-1 Prachuap B, Year 2018.tif.pred.tif"

import geopandas as gpd
import rasterio
from rasterio import features
from sklearn import metrics

ground_truth = gpd.read_file(ground_truth_file)
geom = [shapes for shapes in ground_truth.geometry]
prediction_result = rasterio.open(prediction_result_file)

ground_truth_rasterized = features.rasterize(geom,
                                out_shape = prediction_result.shape,
                                fill = 0,
                                out = None,
                                transform = prediction_result.transform,
                                all_touched = False,
                                default_value = 1,
                                dtype = None)

ground_truth_rasterized_1d_array = ground_truth_rasterized.ravel()
prediction_result_1d_array = prediction_result.read(1).ravel()

confusion_matrix = metrics.confusion_matrix(ground_truth_rasterized_1d_array, prediction_result_1d_array)
iou =  confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[0,1] + confusion_matrix[1,0])
print("IoU:       %.4f" % iou)

print("Accuracy:  %.4f" % metrics.accuracy_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))
print("Precision: %.4f" % metrics.precision_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))
print("Recall:    %.4f" % metrics.recall_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))


ท่านสามารถเรียกดูผลลัพธ์การแบ่งกลุ่ม(segmentation results) ได้โดยการโหลดเอาต์พุต GeoTiff บน QGIS ดาวน์โหลด GeoTiff ภายใต้ .... ด้านล่างนี้คือตัวอย่างที่แสดงเอาต์พุตบน QGIS

ภาพจาก QGIS

ที่นี่ คุณอาจะเข้าใจว่าโมเดลจำเป็นต้องมีกระบวนการเพิ่มเติมเพื่อใช้สำหรับภูมิภาคอื่นๆ

## 3-2  ปรับแต่งโมเดลที่ผ่านการเทรนไปยังภูมิภาคอื่นๆ

ในส่วนย่อยนี้ ท่านจะได้ฝึกฝนการปรับแต่งโมเดลที่ประสิทธิภาพยังไม่ดีใน 3-1 กระบวนการปรับแต่งโมเดลที่ได้รับการเทรนโดยใช้ชุดข้อมูลการเทรนอื่นจากภูมิภาค B เพื่อให้โมเดลสามารถจดจำรูปแบบในภูมิภาค B ได้

ขั้นแรก ให้โหลดโมเดลที่ผ่านการเทรนซึ่งสร้างในส่วนที่ 2-2 ในครั้งนี้ โมเดลถูกโหลดด้วยเอ็นโค้ดเดอร์(encoders)ที่หยุดทำงาน ซึ่งช่วยรักษา weights ในโมเดลที่ฝึกไว้ล่วงหน้า

In [ ]:
## Specify the model trained in Section 2-2 ##
model_file = 'demo_model.hdf5'
##############################################

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm
import tensorflow_addons as tfa
sm.set_framework('tf.keras')

# Set a model architecture and backbone same to the model for loading.
model = sm.FPN('efficientnetb2', classes=1, activation='sigmoid')
# Set a path to the trained model file for testing.
model.load_weights(model_file)

ต่อไป สร้าง training patch images จากภูมิภาค B

In [ ]:
# Configure parameters for patch images.
n_patch = 50
patch_size = 128

in_vec_dir = 'vec/'
in_img_dir = '3_img/'
out_ann_dir = 'ann/'
out_patch_dir = '3-2_patch'

import os, glob, random
import tensorflow as tf
import segmentation_models as sm

os.system('bash semantic_segmentation_training/rasterize_polygon_layers.sh "'
          + in_vec_dir + '" "'
          + in_img_dir + '" "'
          + out_ann_dir +'"'
          )

in_ann_dir = out_ann_dir
os.system('bash semantic_segmentation_training/gen_training_patches.sh "' 
         + in_img_dir + '" "' 
          + in_ann_dir + '" ' 
          + str(patch_size) + ' '
          + str(n_patch) + ' '
          + out_patch_dir
          )


จากนั้นเทรนโมเดลที่โหลดโดยใช้ patch images ที่สร้างขึ้น ลองประมวลผล epoch ด้วยจำนวนน้อย เช่น 5

In [ ]:
# Configure parameters for model training,
n_epoch = 5
batch_size =16
validation_split=0.2
checkpoint_file = '3-2_checkpoint.hdf5'


import numpy as np
from PIL import Image

# generating batches of images and their corresponding masks
def image_generator(files, batch_size = batch_size, sz = (patch_size, patch_size)):
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    
    for f in batch:
        #get the masks. Note that masks are png files 
        try:
          mask = Image.open(f.replace("patch_img","patch_ann"))
        except:
          continue
        mask = np.array(mask.resize(sz))
        batch_y.append(mask)

        #preprocess the raw images 
        raw = Image.open(f)
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
          raw = np.stack((raw,)*3, axis=-1)
        else:
          raw = raw[:,:,0:3]
        
        batch_x.append(raw)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255.
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,axis = 3)

    yield (batch_x, batch_y.astype(np.float32))

# list of all files in the directory specified by patch_img
all_files = glob.glob(out_patch_dir + '/patch_img/*.tif')

# randomly shuffles the order of the files in all_files
random.shuffle(all_files) 

#split into training and testing
split = int((1-validation_split) * len(all_files))
train_files = all_files[0:split]
test_files  = all_files[split:]

train_generator = image_generator(train_files, batch_size = batch_size) # creates a generator for the training set 
test_generator  = image_generator(test_files, batch_size = batch_size) # creates a generator for the testing set 

# retrieves the next batch of images and masks from the training set generator
x, y= next(train_generator)


import tensorflow_addons as tfa
model.compile(
    optimizer = tfa.optimizers.RectifiedAdam(), # tf.keras.optimizers.Adam(),
    loss = sm.losses.DiceLoss(),
    metrics = ['accuracy',sm.metrics.iou_score] 
)

import datetime

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_file,
    save_weights_only=True,
    save_best_only=True)


# performing model training 
import math
train_step = math.ceil(len(glob.glob(out_patch_dir+'/patch_img/*.tif')) * (1 - validation_split) / batch_size)
test_step = math.ceil(len(glob.glob(out_patch_dir+'/patch_img/*.tif')) * validation_split / batch_size)

model_history = model.fit(train_generator, 
                          epochs = n_epoch, 
                          steps_per_epoch = train_step,
                          validation_data = test_generator, 
                          validation_steps = test_step,
                          callbacks = model_checkpoint_callback, 
                          verbose = 1)


ทดสอบโมเดลที่ผ่านการเทรนโดยปรับแต่งกับชุดข้อมูลใหม่

In [ ]:
## Specify the model trained in Section 3-2 ##
model_file = '3-2_checkpoint.hdf5'
##############################################

input_geotiff = '3_img/Prachuap B, Year 2018.tif'
output_pred_file = 'test_results/3-2 Prachuap B, Year 2018.tif.pred.tif'
test_patch_size = 256

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

# Set a model architecture and backbone same to the model for loading.
model = sm.FPN('efficientnetb2', classes=1, activation='sigmoid')
# Set a path to the trained model file for testing.
model.load_weights(model_file)

import os, tempfile, shutil, glob
import numpy as np
from PIL import Image

workdir = tempfile.mkdtemp()
os.system('bash semantic_segmentation_training/split_images_for_testing.sh "' 
          + input_geotiff + '" ' 
          + workdir + ' ' 
          + str(test_patch_size) )

os.makedirs(workdir + '/pat_prd/', exist_ok=True)
for f in glob.glob(workdir + '/pat_img/*.tif'):
  raw = Image.open(f)
  out_patch_size = raw.size[0]
  raw = np.array(raw)
  raw = np.array(raw)/255.
  raw = raw[:,:,0:3]

  pred = model.predict(np.expand_dims(raw, 0))
  pred[pred >= 0.5] = 1
  pred[pred < 0.5] = 0
  pred = pred.astype(np.uint8).reshape([out_patch_size,out_patch_size])
  im = Image.fromarray(pred)
  im.save(workdir + '/pat_prd/' + os.path.basename(f) + '.pred.tif')

os.system('bash semantic_segmentation_training/merge_annotation_for_testing.sh ' 
          + workdir + ' "' 
          + input_geotiff + '" "' 
          + output_pred_file + '"')


สุดท้าย คำนวณ perfomance indicators เช่น ความแม่นยำ(accuracy)และ IoU เพื่อเปรียบเทียบกับผลการทดสอบใน 3-1

In [ ]:
ground_truth_file = "vec/Rubber plantation, Year 2018.gpkg"
prediction_result_file = "test_results/3-2 Prachuap B, Year 2018.tif.pred.tif"

import geopandas as gpd
import rasterio
from rasterio import features
from sklearn import metrics

ground_truth = gpd.read_file(ground_truth_file)
geom = [shapes for shapes in ground_truth.geometry]
prediction_result = rasterio.open(prediction_result_file)

ground_truth_rasterized = features.rasterize(geom,
                                out_shape = prediction_result.shape,
                                fill = 0,
                                out = None,
                                transform = prediction_result.transform,
                                all_touched = False,
                                default_value = 1,
                                dtype = None)

ground_truth_rasterized_1d_array = ground_truth_rasterized.ravel()
prediction_result_1d_array = prediction_result.read(1).ravel()

confusion_matrix = metrics.confusion_matrix(ground_truth_rasterized_1d_array, prediction_result_1d_array)
iou =  confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[0,1] + confusion_matrix[1,0])
print("IoU:       %.4f" % iou)

print("Accuracy:  %.4f" % metrics.accuracy_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))
print("Precision: %.4f" % metrics.precision_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))
print("Recall:    %.4f" % metrics.recall_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))


ท่านสามารถเรียกดูผลลัพธ์การแบ่งกลุ่ม(segmentation results) ได้โดยการโหลดเอาต์พุต GeoTiff บน QGIS ดาวน์โหลด GeoTiff ภายใต้ .... ด้านล่างนี้คือตัวอย่างที่แสดงเอาต์พุตบน QGIS

ภาพจาก QGIS

ท่านอาจเห็นประสิทธิภาพของโมเดลที่ดีกว่าการทดสอบใน 3-1 นี่เป็นผลลัพธ์ของกระบวนการปรับแต่ง ซึ่งปรับปรุงประสิทธิภาพของโมเดลแม้ว่าจะมี epochs จำนวนน้อยก็ตาม

## 3-3 เทรนโมเดลโดยรวมกับชุดข้อมูลการเทรน
ในส่วนย่อยนี้ ท่านมีการทดลองการเทรนโมเดลโดยใช้ชุดข้อมูลจากภูมิภาค A และภูมิภาค B โมเดลที่ผ่านการเทรนคาดหวังว่าจะทำงานได้ดีสำหรับทั้งสองภูมิภาค เนื่องจากโมเดลได้เรียนรู้รูปแบบของภูมิภาค อย่างไรก็ตาม ท่านจะเห็นว่ามันต้องใช้การวนซ้ำและ epochs เยอะกว่าจะได้ประสิทธิภาพที่ดีกว่าการปรับแต่งแบบใน 3-2

สร้าง training patch images จากทั้งสองภูมิภาค ติดตั้งสอง GeoTiffs สำหรับการสร้าง patch images

In [ ]:
!rm -rf 3-3_patch

In [ ]:
# Configure parameters for patch images.
n_patch = 25
patch_size = 128

in_vec_dir = 'vec/'
in_img_dir = '3-3_train_img/'
out_ann_dir = 'ann/'
out_patch_dir = '3-3_patch'

import os, glob, random
import tensorflow as tf
import segmentation_models as sm

os.system('bash semantic_segmentation_training/rasterize_polygon_layers.sh "'
          + in_vec_dir + '" "'
          + in_img_dir + '" "'
          + out_ann_dir +'"'
          )

in_ann_dir = out_ann_dir
os.system('bash semantic_segmentation_training/gen_training_patches.sh "' 
         + in_img_dir + '" "' 
          + in_ann_dir + '" ' 
          + str(patch_size) + ' '
          + str(n_patch) + ' '
          + out_patch_dir
          )


จากนั้นเทรนโมเดลโดยใช้ patch images ที่สร้างขึ้นตั้งแต่เริ่มต้น เราเทรนไว้เพียง 5 epochs เพื่อเปรียบเทียบกับโมเดลใน 3-2

In [ ]:
# Configure parameters for model training,
n_epoch = 5
batch_size =16
validation_split=0.2
checkpoint_file = '3-3_checkpoint_epoch_5.hdf5'


import numpy as np
from PIL import Image

# generating batches of images and their corresponding masks
def image_generator(files, batch_size = batch_size, sz = (patch_size, patch_size)):
  while True: 
    
    #extract a random batch 
    batch = np.random.choice(files, size = batch_size)    
    
    #variables for collecting batches of inputs and outputs 
    batch_x = []
    batch_y = []
    
    for f in batch:
        #get the masks. Note that masks are png files 
        try:
          mask = Image.open(f.replace("patch_img","patch_ann"))
        except:
          continue
        mask = np.array(mask.resize(sz))
        batch_y.append(mask)

        #preprocess the raw images 
        raw = Image.open(f)
        raw = raw.resize(sz)
        raw = np.array(raw)

        #check the number of channels because some of the images are RGBA or GRAY
        if len(raw.shape) == 2:
          raw = np.stack((raw,)*3, axis=-1)
        else:
          raw = raw[:,:,0:3]
        
        batch_x.append(raw)

    #preprocess a batch of images and masks 
    batch_x = np.array(batch_x)/255.
    batch_y = np.array(batch_y)
    batch_y = np.expand_dims(batch_y,axis = 3)

    yield (batch_x, batch_y.astype(np.float32))

# list of all files in the directory specified by patch_img
all_files = glob.glob(out_patch_dir + '/patch_img/*.tif')

# randomly shuffles the order of the files in all_files
random.shuffle(all_files) 

#split into training and testing
split = int((1-validation_split) * len(all_files))
train_files = all_files[0:split]
test_files  = all_files[split:]

train_generator = image_generator(train_files, batch_size = batch_size) # creates a generator for the training set 
test_generator  = image_generator(test_files, batch_size = batch_size) # creates a generator for the testing set 

# retrieves the next batch of images and masks from the training set generator
x, y= next(train_generator)


import tensorflow_addons as tfa
model.compile(
    optimizer = tfa.optimizers.RectifiedAdam(), # tf.keras.optimizers.Adam(),
    loss = sm.losses.DiceLoss(),
    metrics = ['accuracy',sm.metrics.iou_score] 
)

import datetime

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_file,
    save_weights_only=True,
    save_best_only=True)


# performing model training 
import math
train_step = math.ceil(len(glob.glob(out_patch_dir+'/patch_img/*.tif')) * (1 - validation_split) / batch_size)
test_step = math.ceil(len(glob.glob(out_patch_dir+'/patch_img/*.tif')) * validation_split / batch_size)

model_history = model.fit(train_generator, 
                          epochs = n_epoch, 
                          steps_per_epoch = train_step,
                          validation_data = test_generator, 
                          validation_steps = test_step,
                          callbacks = model_checkpoint_callback, 
                          verbose = 1)


ลองทดสอบโมเดลที่ผ่านการเทรนไปยังภูมิภาค B เนื่องจากโมเดลไม่ได้รับการเทรนใน epochs ที่เพียงพอ ประสิทธิภาพจึงน้อยกว่าโมเดลใน 3-1

In [ ]:
## Specify the model trained in Section 3-2 ##
model_file = '3-3_checkpoint_epoch_5.hdf5'
##############################################

input_geotiff = '3_img/Prachuap B, Year 2018.tif'
output_pred_file = 'test_results/3-3 epoch_5 Prachuap B, Year 2018.tif.pred.tif'
test_patch_size = 256

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm
import tensorflow_addons as tfa
sm.set_framework('tf.keras')

# Set a model architecture and backbone same to the model for loading.
model = sm.FPN('efficientnetb2', classes=1, activation='sigmoid')
# Set a path to the trained model file for testing.
model.load_weights(model_file)

import os, tempfile, shutil, glob
import numpy as np
from PIL import Image
os.environ['SM_FRAMEWORK'] = "tf.keras"
import segmentation_models as sm
sm.set_framework('tf.keras')

workdir = tempfile.mkdtemp()
os.system('bash semantic_segmentation_training/split_images_for_testing.sh "' 
          + input_geotiff + '" ' 
          + workdir + ' ' 
          + str(test_patch_size) )

os.makedirs(workdir + '/pat_prd/', exist_ok=True)
for f in glob.glob(workdir + '/pat_img/*.tif'):
  raw = Image.open(f)
  out_patch_size = raw.size[0]
  raw = np.array(raw)
  raw = np.array(raw)/255.
  raw = raw[:,:,0:3]

  pred = model.predict(np.expand_dims(raw, 0))
  pred[pred >= 0.5] = 1
  pred[pred < 0.5] = 0
  pred = pred.astype(np.uint8).reshape([out_patch_size,out_patch_size])
  im = Image.fromarray(pred)
  im.save(workdir + '/pat_prd/' + os.path.basename(f) + '.pred.tif')

os.system('bash semantic_segmentation_training/merge_annotation_for_testing.sh ' 
          + workdir + ' "' 
          + input_geotiff + '" "' 
          + output_pred_file + '"')


In [ ]:
ground_truth_file = "vec/Rubber plantation, Year 2018.gpkg"
prediction_result_file = "test_results/3-3 epoch_5 Prachuap B, Year 2018.tif.pred.tif"

import geopandas as gpd
import rasterio
from rasterio import features
from sklearn import metrics

ground_truth = gpd.read_file(ground_truth_file)
geom = [shapes for shapes in ground_truth.geometry]
prediction_result = rasterio.open(prediction_result_file)

ground_truth_rasterized = features.rasterize(geom,
                                out_shape = prediction_result.shape,
                                fill = 0,
                                out = None,
                                transform = prediction_result.transform,
                                all_touched = False,
                                default_value = 1,
                                dtype = None)

ground_truth_rasterized_1d_array = ground_truth_rasterized.ravel()
prediction_result_1d_array = prediction_result.read(1).ravel()

confusion_matrix = metrics.confusion_matrix(ground_truth_rasterized_1d_array, prediction_result_1d_array)
iou =  confusion_matrix[1,1] / (confusion_matrix[1,1] + confusion_matrix[0,1] + confusion_matrix[1,0])
print("IoU:       %.4f" % iou)

print("Accuracy:  %.4f" % metrics.accuracy_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))
print("Precision: %.4f" % metrics.precision_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))
print("Recall:    %.4f" % metrics.recall_score(ground_truth_rasterized_1d_array, prediction_result_1d_array))


ต่อไป มาเทรนโมเดลสำหรับ ***** epochs กัน โมเดลจะมีประสิทธิภาพดีกว่าโมเดลที่ได้รับการฝึกฝนด้วยแค่ 5 epochs 

In [ ]:
##

ทดสอบโมเดลที่ผ่านการเทรนไปยังภูมิภาค B ท่านอาจเห็นประสิทธิภาพที่ดีกว่าโมเดลที่ผ่านการฝึกอบรมเพียงแค่ 5 epochs 

In [ ]:
##

โมเดลที่ผ่านการเทรนปฏิบัติการได้น้อยลงใน epochs จำนวนเท่ากัน ซึ่งโมเดลที่ปรับแต่งแล้วนั้นมีประสิทธิภาพที่ใช้งานได้จริง การเปรียบเทียบเผยให้เห็นว่าการปรับแต่งประหยัดเวลาสำหรับการเทรนโมเดลที่เหมาะสมสำหรับภูมิภาค B

หากท่านสามารถเผื่อเวลาไว้มากสำหรับการเทรนโมเดล การเทรนตั้งแต่เริ่มต้นคือตัวเลือกในการพัฒนาโมเดล อย่างไรก็ตาม อาจทำให้เกิดค่าใช้จ่ายสูงเมื่อท่านขยายการดำเนินงานไปยังภูมิภาคกว้างๆ